In [ ]:
!pip install d2l
import torch
from torch import nn
from d2l import torch as d2l

batch_size, max_window_size, num_noise_words = 512, 5, 5
data_iter, vocab = d2l.load_data_ptb(batch_size, max_window_size,
                                     num_noise_words)

In [2]:
embed = nn.Embedding(num_embeddings=20, embedding_dim=4)
print(f'Parameter embedding_weight ({embed.weight.shape}, '
      'dtype={embed.weight.dtype})')

Parameter embedding_weight (torch.Size([20, 4]), dtype={embed.weight.dtype})


In [3]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
embed(x)

tensor([[[-0.4556,  0.3750,  1.2285, -1.7191],
         [-0.3562,  0.1198,  0.0570, -0.7856],
         [ 2.6621,  0.8120, -0.3307, -0.1548]],

        [[ 0.0323,  1.2174, -0.4587,  0.2087],
         [ 0.5750,  1.2456,  1.7160,  1.7760],
         [ 0.2685, -0.7677, -1.4447, -1.4716]]], grad_fn=<EmbeddingBackward>)

**Implemented skip-gram to CBOW**

In [ ]:
def skip_gram(center, contexts_and_negatives, embed_v, embed_u):
    v = embed_v(center)
    u = embed_u(contexts_and_negatives)
    pred = torch.bmm(v, u.permute(0, 2, 1))
    return pred

In [ ]:
skip_gram(torch.ones((2, 1), dtype=torch.long),
          torch.ones((2, 4), dtype=torch.long), embed, embed).shape

**CBOW**

In [5]:
def CBOW(contexts, center_negative, masks, embed_v, embed_u):
    v = embed_v(contexts).sum(dim=1) /  masks.sum(dim=-1).view(-1, 1)
    u = embed_u(center_negative)
    pred = torch.bmm(v[:, None, :], u.permute(0, 2, 1))
    return pred